# Write-up and code for Mar 6

## To Do:
- ~~Write Proof (with precise notation) of the Policy Gradient Theorem~~
- ~~Derive the score function for softmax policy (for finite set of actions)~~
- ~~Derive the score function for gaussian policy (for continuous actions)~~
- ~~Write code for the REINFORCE Algoithm (Monte-Carlo Policy Gradient Algorithm, i.e., no Critic)~~
- Optional: Write code for Actor-Critic Policy Gradient Algorithms (with TD(0) and with Eligiblity-Traces-based TD(Lambda))
- ~~Write Proof (with proper notation) of the Compatible Function Approximation Theorem~~
- Optional: Write code for Natural Policy Gradient Algorithm based on Compatible Linear Function Appeoximation for Critic

## Proof of Policy Gradient Theorem

The policy $\pi$ is a function of both the state and the parameter $\theta$ hence we have $\pi(a~|~s, \theta)$. Take the gradient of the state-value function with respect to the parameter $\theta$

$$
J(\theta) = \int_{\mathcal S}p_0(s_o)\cdot v_\pi(s_0)\cdot ds_0 = \int_{\mathcal S}p_0(s_o)\cdot \int_{\mathcal A}\pi(a_0, s_0; \theta) \cdot q_\pi(s_0,a_0)\cdot da_0 \cdot ds_0
$$

Calculate $\nabla J(\theta)$ by parts $\pi(a_0, s_0;\theta)$ and $q_\pi(s_0, a_0)$

$$
\begin{align}
\nabla J(\theta) & = \int_{\mathcal S}p_0(s_o)\cdot \int_{\mathcal A}\nabla\pi(a_0, s_0; \theta) \cdot q_\pi(s_0,a_0)\cdot da_0 \cdot ds_0\\
& + \int_{\mathcal S}p_0(s_o)\cdot \int_{\mathcal A}\pi(a_0, s_0; \theta) \cdot \nabla q_\pi(s_0,a_0)\cdot da_0 \cdot ds_0\\
\\
& \text{Use Bellman to expand } q_\pi\\
\\
& = \int_{\mathcal S}p_0(s_o)\cdot \int_{\mathcal A}\nabla\pi(a_0, s_0; \theta) \cdot q_\pi(s_0,a_0)\cdot da_0 \cdot ds_0 \\
& + \int_{\mathcal S}p_0(s_o)\cdot \int_{\mathcal A}\pi(a_0, s_0; \theta) \cdot \nabla  \Big(\mathcal R_{s_0}^{a_0} + \int_{\mathcal S}\gamma\mathcal P_{s_0,s_1}^{a_0}\cdot v_\pi(s_1)\cdot ds_1 \Big) \cdot da_0 \cdot ds_0\\
\\
& \text{The gradient w.r.t. the reward is 0}\\
\\
& = \int_{\mathcal S}p_0(s_o)\cdot \int_{\mathcal A}\nabla\pi(a_0, s_0; \theta) \cdot q_\pi(s_0,a_0)\cdot da_0 \cdot ds_0 \\
& + \int_{\mathcal S}p_0(s_o)\cdot \int_{\mathcal A}\pi(a_0, s_0; \theta) \cdot \nabla  \Big(\int_{\mathcal S}\gamma\mathcal P_{s_0,s_1}^{a_0}\cdot v_\pi(s_1)\cdot ds_1 \Big) \cdot da_0 \cdot ds_0\\
\\
& \text{Only the value function depends on $\theta$}\\
\\
& = \int_{\mathcal S}p_0(s_o)\cdot \int_{\mathcal A}\nabla\pi(a_0, s_0; \theta) \cdot q_\pi(s_0,a_0)\cdot da_0 \cdot ds_0 \\
& + \int_{\mathcal S}p_0(s_o)\cdot \int_{\mathcal A}\pi(a_0, s_0; \theta) \cdot \int_{\mathcal S}\gamma\mathcal P_{s_0,s_1}^{a_0}\cdot \nabla v_\pi(s_1)\cdot ds_1 \cdot da_0 \cdot ds_0 \\
\\
& \text{Note that } \int_{\mathcal A} \pi(s_0, a_0;\theta)\mathcal P_{s_0,s_1}^{a_0} \cdot da_0 = Pr(s_0\rightarrow s_1, 1, \pi) \\
& \text{which is the probability of transitioning from $s_0$ to $s_1$ in one step under policy $\pi$}\\
\\
& = \int_{\mathcal S}p_0(s_o)\cdot \int_{\mathcal A}\nabla\pi(a_0, s_0; \theta) \cdot q_\pi(s_0,a_0)\cdot da_0 \cdot ds_0 \\
& + \int_{\mathcal S}\big( \int_{\mathcal S}\gamma \cdot p_0(s_o) \cdot Pr(s_0\rightarrow s_1, 1, \pi)\cdot ds_0 \big)\nabla v_\pi(s_1)\cdot ds_1 \\
\\
& \text{Once again, expand $v_\pi$}\\
\\
& = \int_{\mathcal S}p_0(s_o)\cdot \int_{\mathcal A}\nabla\pi(a_0, s_0; \theta) \cdot q_\pi(s_0,a_0)\cdot da_0 \cdot ds_0 \\
& + \int_{\mathcal S}\big( \int_{\mathcal S}\gamma \cdot p_0(s_o) \cdot Pr(s_0\rightarrow s_1, 1, \pi)\cdot ds_0 \big)\nabla \Big(\int_{\mathcal A} \pi(a_1, s_1; \theta) \cdot q_\pi(s_1,a_1)\cdot da_1 \Big)\cdot ds_1 \\
\\
& \text{We are now back to a similar scenario as where we started, where we need to take the gradient by parts on $\pi$ and $q_\pi$}\\
& \text{telescoping these equations to infinity gives us}\\
\\
& = \sum_{t=0}^\infty \int_{\mathcal S}\int_{\mathcal S}\gamma^t\cdot p_0(s_0)\cdot Pr(s_0\rightarrow s_t, t, \pi)\cdot ds_0\int_{\mathcal A}\nabla \pi(s_t,a_t;\theta)\cdot q_\pi(s_t, a_t) \cdot da_t\cdot ds_t\\
\\
& = \int_{\mathcal S}\int_{\mathcal S}\sum_{t=0}^\infty \gamma^t\cdot p_0(s_0)\cdot Pr(s_0\rightarrow s, t, \pi)\cdot ds_0\int_{\mathcal A}\nabla \pi(s,a;\theta)\cdot q_\pi(s, a) \cdot da\cdot ds\\
\\
& \text{Note that by definition $\int_{\mathcal S}\sum_{t=0}^\infty \gamma^t\cdot p_0(s_0)\cdot Pr(s_0\rightarrow s, t, \pi)\cdot ds_0 = \rho_\pi(s)$}\\
\\
& = \int_{\mathcal S}\rho_\pi(s)\int_{\mathcal A}\nabla \pi(s,a;\theta)\cdot q_\pi(s, a) \cdot da\cdot ds\\
\end{align}
$$


## Derive the score function for softmax policy

The score function is defined as $\nabla_\theta \log \pi(s,a;\theta)$. A softmax policy is given by

$$
\pi(s,a;\theta) = \frac{e^{\theta^\top \phi(s,a)}}{\sum_{a'} e^{\theta^\top \phi(s,a')}}
$$

Find the score function of a softmax policy

$$
\begin{align}
\nabla_\theta \log\pi(s,a;\theta) & = \nabla_\theta \log \Big(\frac{e^{\theta^\top \phi(s,a)}}{\sum_{a'} e^{\theta^\top \phi(s,a')}}\Big) \\
\\
& = \nabla_\theta \log e^{\theta^\top \phi(s,a)} - \nabla_\theta \log \sum_{a'} e^{\theta^\top \phi(s,a')}\\
\\
& = \phi(s,a) - \frac{1}{\sum_{a'} e^{\theta^\top \phi(s,a')}}\cdot \sum_{a''} e^{\theta^\top \phi(s,a'')} \cdot \phi(s,a'')\\
\\
& = \phi(s,a) - \sum_{a''} \frac{e^{\theta^\top \phi(s,a'')}}{\sum_{a'} e^{\theta^\top \phi(s,a')}} \cdot \phi(s,a'')\\
\\
& = \phi(s,a) - \sum_{a''} \pi(s,a'';\theta)\cdot\phi(s,a'')\\
\\
& = \phi(s,a) - \mathbb E_\pi[\phi(s,\cdot)]
\end{align}
$$

## Derive the score function for Gaussian policy
A Gaussian policy is given by

$$
a \sim \mathcal N(\theta^\top\cdot\phi(s), \sigma^2),~~ \forall s \in \mathcal S
$$

Hence,

$$
\pi(s,a;\theta) = \frac{1}{\sqrt{2\pi}\sigma}\exp\big(-\frac{(a - \theta^\top\cdot\phi(s))^2}{2\sigma^2}\big)
$$

Find the score function of a Gaussian policy

$$
\begin{align}
\nabla_\theta \log\pi(s,a;\theta) & = \nabla_\theta \log \Big(\frac{1}{\sqrt{2\pi}\sigma}\exp\big(-\frac{(a - \theta^\top\cdot\phi(s))^2}{2\sigma^2}\big)\Big) \\
\\
& = \nabla_\theta \Big(-\frac{(a - \theta^\top\cdot\phi(s))^2}{2\sigma^2} \Big)\\
\\
& = \frac{a - \theta^\top\cdot\phi(s)}{\sigma^2}\cdot \phi(s)
\end{align}
$$

## REINFORCE Algoithm (Monte-Carlo Policy Gradient Algorithm)

In [3]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [5]:
import numpy as np
from modules.RL_interface_FA import RL_interface_FA
from modules.state_action_vars import S, A
from typing import Callable, List, Tuple

In [8]:
def reinforce(sampler: RL_interface_FA, polf: Callable[[S, np.ndarray], A], 
              polf_grad: Callable[[S, A, np.ndarray], np.ndarray], d: int,
              num_epi: int, num_steps: int, alpha: float) -> np.ndarray:
    # reinforce algorithm
    
    # initialize theta, d is the number of parameters
    theta = np.zeros((d,1))
    # retrieve discount factor
    gamma = sampler.gamma
    
    for i in range(num_epi):
        s_list, a_list, r_list = generate_episode(sampler, polf, theta, num_steps)
        for t in range(num_steps+1):
            G = np.dot(np.power(gamma, np.arange(num_steps + 1 - t)), np.array(r_list[t:]))
            theta += alpha*np.power(gamma,t)*polf_grad(s_list[t], a_list[t], theta)
            
    return theta

def generate_episode(sampler: RL_interface_FA, polf: Callable[[S, np.ndarray], A], 
                     theta: np.ndarray, num_steps: int) \
        -> Tuple[List[S], List[A], List[float]]:
    # generate one episode
    s = sampler.init_state_gen()
    s_list = [s]
    a = polf(s, theta)
    a_list = [a]
    r_list = []
    for t in range(num_steps):
        sp, r = sampler.state_reward_func(s, a)
        ap = polf(sp, theta)
        
        s_list.append(sp)
        a_list.append(ap)
        r_list.append(r)
        s, a = sp, ap
        
    _, r = sampler.state_reward_func(s, a)
    r_list.append(r)
    
    return s_list, a_list, r_list

## Proof of Compatible Function Approximation Theorem
__Theorem:__

__1.__ Critic gradient is compatible with the Actor score function

$$
\nabla_w Q(s,a;w) = \nabla_\theta \log\pi(s,a;\theta)
$$

__2.__ Critic parameters $w$ minimize the following mean-squared error:

$$
\epsilon = \int_{\mathcal S}\rho^\pi(s)\int_{\mathcal A} \pi(s,a;\theta)\big(Q^\pi(s,a)-Q(s,a;w)\big)^2\cdot da\cdot ds
$$

Then the Policy Gradient using critic Q(s,a;w) is exact:

$$
\nabla_\theta J(\theta) = \int_{\mathcal S}\rho^\pi(s)\int_{\mathcal A}\nabla_\theta \pi(s,a;\theta)\cdot Q(s,a;w)\cdot da\cdot ds
$$

__Proof:__

For $w$ that minimizes $\epsilon$ we have that

$$
\int_{\mathcal S}\rho^\pi(s)\int_{\mathcal A} \pi(s,a;\theta)\big(Q^\pi(s,a)-Q(s,a;w)\big)\nabla_w Q(s,a;w)\cdot da\cdot ds = 0
$$

Since $\nabla_w Q(s,a;w) = \nabla_\theta \log \pi(s,a;\theta)$ we get

$$
\int_{\mathcal S}\rho^\pi(s)\int_{\mathcal A} \pi(s,a;\theta)\big(Q^\pi(s,a)-Q(s,a;w)\big)\nabla_\theta \log \pi(s,a;\theta)\cdot da\cdot ds = 0
$$

Which then gives

$$
\int_{\mathcal S}\rho^\pi(s)\int_{\mathcal A} \pi(s,a;\theta)\cdot Q^\pi(s,a)\nabla_\theta \log \pi(s,a;\theta)\cdot da\cdot ds = \int_{\mathcal S}\rho^\pi(s)\int_{\mathcal A} \pi(s,a;\theta)\cdot Q(s,a;w)\nabla_\theta \log \pi(s,a;\theta)\cdot da\cdot ds 
$$

But 

$$
\nabla_\theta J(\theta) = \int_{\mathcal S}\rho^\pi(s)\int_{\mathcal A} \pi(s,a;\theta)\cdot Q^\pi(s,a)\nabla_\theta \log \pi(s,a;\theta)\cdot da\cdot ds \\
$$

So

$$
\begin{align}
\nabla_\theta J(\theta) & = \int_{\mathcal S}\rho^\pi(s)\int_{\mathcal A} \pi(s,a;\theta)\cdot Q(s,a;w)\nabla_\theta \log \pi(s,a;\theta)\cdot da\cdot ds \\
& = \int_{\mathcal S}\rho^\pi(s)\int_{\mathcal A} \nabla_\theta \pi(s,a;\theta)\cdot Q(s,a;w)\cdot da\cdot ds
\end{align}
$$